In [2]:
import os
import getpass

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")

#for tracing
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "RAG Development"

### INDEXING

In [14]:
question = "Who is Escanor?"
document = "Escanor is a fictional character from the manga and anime series 'The Seven Deadly Sins' (Nanatsu no Taizai), created by Nakaba Suzuki. He is known as the Lion's Sin of Pride and is a member of the titular group, the Seven Deadly Sins, who are a band of powerful knights. Escanor is renowned for his immense strength, which varies dramatically between day and night due to his unique ability, 'Sunshine.' During the day, especially at noon when the sun is at its peak, Escanor transforms into a nearly invincible warrior with overwhelming power and unshakable confidence, embodying the epitome of pride. Conversely, at night, he becomes a frail and timid man, significantly weaker and lacking in self-assurance. This duality makes Escanor a compelling and multifaceted character within the series. His prideful and noble nature, coupled with his tragic and complex backstory, has made him a favorite among fans of 'The Seven Deadly Sins.'"

In [15]:
import tiktoken

def num_token(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens , encoding

num_token(question, "cl100k_base")

(6, <Encoding 'cl100k_base'>)

In [16]:
from langchain_openai import OpenAIEmbeddings
embed = OpenAIEmbeddings()
question_result = embed.embed_query(question)
document_result = embed.embed_query(document)
len(question_result)            #Generates a fixed lenght vector representation

1536

In [17]:
#employing cosine similarity

import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product/ (norm_vec1* norm_vec2)
similarity = cosine_similarity(question_result, document_result)
print(similarity)

0.8933074349631601


###### Loading a document

In [40]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_paths=("https://medium.com/@onyeocha.keside/understanding-bert-the-game-changer-in-natural-language-processing-47ca6ba78cc9/",), bs_kwargs = dict(parse_only =bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
medium_post = loader.load()

In [41]:
medium_post

[Document(page_content='', metadata={'source': 'https://medium.com/@onyeocha.keside/understanding-bert-the-game-changer-in-natural-language-processing-47ca6ba78cc9/'})]

###### Split

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300,
    chunk_overlap = 50)
splits = splitter.split_documents(medium_post)

print(f"Number of splits: {len(splits)}")


Number of splits: 0


##### INDEX

In [38]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding = OpenAIEmbeddings())


retriever = vectorstore.as_retriever()

ValueError: Expected IDs to be a non-empty list, got 0 IDs